MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

Interacting with the MLflow tracking server
The MlflowClient object allows us to interact with...

an MLflow Tracking Server that creates and manages experiments and runs.
an MLflow Registry Server that creates and manages registered models and model versions.
To instantiate it we need to pass a tracking URI and/or a registry URI



In [ ]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()


In [ ]:
client.create_experiment(name="my-cool-experiment")

In [ ]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)


In [ ]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")


Interacting with the Model Registry
In this section We will use the MlflowClient instance to:

Register a new version for the experiment nyc-taxi-regressor
Retrieve the latests versions of the model nyc-taxi-regressor and check that a new version 4 was created.
Transition the version 4 to "Staging" and adding annotations to it.

In [ ]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [ ]:
run_id = "b8904012c84343b5bf8ee72aa8f0f402"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

In [ ]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

In [ ]:
model_version = 4
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

In [ ]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

Comparing versions and selecting the new "Production" model
In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
Download the DictVectorizer that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
Preprocess the test set using the DictVectorizer so we can properly feed the regressors.
Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
Based on the results, update the "Production" model version accordingly.
Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.